In [1]:
from modules.clients import HuggingFaceClient
from modules.miner import get_fix_commits

In [3]:
def download_split(client: HuggingFaceClient,offset: int):
    method = 'rows'
    config = {
        'dataset' : "bigcode/commitpackft",
        'config' : 'javascript',
        'split' : 'train',
        'offset' : offset,
        'length' : 100,
    }
    return client.get(method,config)

train_inputs = []
train_labels = []
num_samples = 0
offset = 2500
while(True):
    hfc = HuggingFaceClient()
    response = download_split(hfc,offset)
    if response.status_code != 200:
        print(response.json())
        break

    data_df = get_fix_commits(response.json())
    num_samples += len(data_df.index)
    print(f"Aquired {len(data_df.index)} samples.\nTotal samples: {num_samples}")
    train_inputs.append(data_df['old_contents'].tolist())
    train_labels.append(data_df['new_contents'].tolist())
    offset += 100

len(train_inputs), len(train_labels)

Aquired 35 samples.
Total samples: 35
Aquired 22 samples.
Total samples: 57
Aquired 27 samples.
Total samples: 84
Aquired 28 samples.
Total samples: 112


KeyboardInterrupt: 

In [6]:
import json
with open('dataset-demo.json','r')as f:
    response =  json.load(f)
response

data_df = get_fix_commits(response)
print(f"Aquired {data_df.index} samples.")
train_inputs = data_df['old_contents'].tolist()
train_labels = data_df['new_contents'].tolist()
offset += 100

len(train_inputs), len(train_labels)

Aquired Index([ 3,  7, 10, 13, 17, 19, 29, 31, 33, 34, 39, 45, 48, 54, 55, 56, 57, 58,
       60, 64, 68, 71, 76, 79, 81, 84, 85, 86, 87, 97, 98],
      dtype='int64') samples.


(31, 31)

In [31]:
import pandas as pd
from googletrans import Translator


dataset_list = []
for item in response['rows']:
        dataset_list.append(item['row'])

df = pd.DataFrame(dataset_list)
patterns_EN = ['fix','bug','patch','repair','refactor','address','error','conflict','revert']
patterns_CN = ['修正','错误','补丁','修理','重构','地址','错误','冲突','还原']
patterns_RU = ['исправить', 'ошибка', 'исправление', 'исправить', 'рефакторинг', 'адрес', 'ошибка', 'конфликт', 'отменить']
patterns_JP = ['修正する','バグ','パッチ','修理する','リファクタリング','アドレス','エラー','競合','復元する']
patterns_SP = ['arreglar','error','parche','reparar','refactorizar','direccionar','error','conflicto','revertir']
patterns_FR = ['corriger','erreur','pilote','réparer','refactoriser','adresse','erreur','conflit','inverser']
patterns_DE = ['korrigieren','Fehler','Patch','reparieren','refaktorisieren','Adresse','Fehler','Konflikt','zurücksetzen']
patterns = patterns_EN + patterns_CN + patterns_RU + patterns_JP + patterns_SP + patterns_FR + patterns_DE
len(df[df['message'].str.contains('|'.join(patterns))].index)

26